<a href="https://colab.research.google.com/github/acucenarodrigues1998/TutorialPyBR2023-GenAINotebooks/blob/main/Tutorial_PyBR_XX_Question_Answering_com_LLM_e_banco_de_vetores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Large Language Models (LLMs)**
As redes Large Language Models, ou modelos de linguagem de grande escala, são um tipo de modelo de aprendizado de máquina que foram desenvolvidos para processar e gerar texto natural em grande quantidade. Eles são treinados em vastos conjuntos de dados textuais, geralmente incluindo uma ampla variedade de textos da internet, como artigos, livros, sites da web e muito mais. Esses modelos são projetados para entender e gerar texto de maneira muito semelhante à linguagem humana, tornando-os úteis em uma variedade de tarefas de processamento de linguagem natural, como tradução automática, resumo de texto, geração de texto, classificação de texto e muito mais.

Aqui está uma explicação simplificada de como esses modelos funcionam:

1. **Arquitetura**: Os modelos de linguagem de grande escala geralmente usam arquiteturas de redes neurais profundas, como transformers. Essas arquiteturas são projetadas para lidar com sequências de palavras em texto, capturando as relações entre as palavras e os contextos em que elas aparecem.

2. **Treinamento**: Para treinar esses modelos, uma grande quantidade de texto é alimentada à rede neural. O treinamento envolve a previsão da próxima palavra em uma sequência de palavras, com base no contexto anterior. O modelo ajusta seus parâmetros internos (pesos) para minimizar a discrepância entre suas previsões e o texto real.

3. **Representações de palavras**: Durante o treinamento, o modelo aprende representações vetoriais de palavras (word embeddings) que capturam o significado e as relações entre as palavras. Isso permite que o modelo compreenda o contexto e a semântica das palavras em um texto.

4. **Geração de texto**: Após o treinamento, o modelo pode ser usado para gerar texto. Dada uma entrada inicial (por exemplo, uma frase ou palavra), o modelo prevê as palavras subsequentes com base em seu conhecimento do contexto e das relações entre as palavras. Ele gera texto que é coerente e gramaticalmente correto.

5. **Transferência de conhecimento**: Além da geração de texto, esses modelos podem ser ajustados (fine-tuning) para tarefas específicas, como classificação de sentimentos, tradução automática ou qualquer outra tarefa de processamento de linguagem natural. Isso é possível porque o modelo pré-treinado já possui um conhecimento linguístico geral que pode ser útil em tarefas mais específicas.

6. **Reforço com Feedback Humano (RLHF)**: Para melhorar ainda mais o desempenho em tarefas específicas, os modelos podem ser ajustados com RLHF, recebendo feedback humano que ajuda a refinar sua capacidade de interagir com os usuários e melhorar o desempenho em situações do mundo real.

Redes Large Language Models, como o GPT-3 da OpenAI, têm sido usadas em diversas aplicações, desde assistentes virtuais até geração de texto automatizada e pesquisa em linguística computacional. Eles têm o potencial de melhorar significativamente a capacidade de máquinas para entender e gerar texto natural, embora também apresentem desafios relacionados à ética e ao viés, que precisam ser considerados ao utilizá-los. O RLHF é uma técnica valiosa que ajuda a ajustar esses modelos para tarefas específicas com base no feedback humano.

## Aplicação prática

Para esse tema utilizaremos o modelo LLaMa 2 e um banco de vetores, disponibilizado pela Meta e construiremos uma estrutura de perguntas e respostas utilizando como base de dados algum site da internet.

In [ ]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers --quiet

In [ ]:
from torch import cuda, bfloat16
import transformers

import warnings
warnings.filterwarnings("ignore")

### Preparando os modelos para inferência e tokenização

#### Preparando os modelos

In [ ]:
model_id = 'NousResearch/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [ ]:
# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [ ]:
# begin initializing HF items, you need an access token
model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

Model loaded on cuda:0


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Definindo critérios de parada

In [ ]:
stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

[[1, 29871, 13, 29950, 7889, 29901], [1, 29871, 13, 28956, 13]]

In [ ]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([    1, 29871,    13, 29950,  7889, 29901], device='cuda:0'),
 tensor([    1, 29871,    13, 28956,    13], device='cuda:0')]

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

#### Definindo o pipeline de inferência

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=4000,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
res = generate_text("Compare the personalities of the Bennet sisters from Jane Austen's Pride and Prejudice.")
print(res[0]["generated_text"])

Compare the personalities of the Bennet sisters from Jane Austen's Pride and Prejudice. Unterscheidung der Persönlichkeitsmerkmale der Bennet-Schwestern aus Jane Austens "Pride and Prejudice".

In Jane Austen's novel "Pride and Prejudice," the Bennet sisters are a lively and diverse group of women, each with their own unique personality traits. Here is a comparison of the personalities of the Bennet sisters:

1. Elizabeth Bennet - The eldest Bennet sister, Elizabeth is intelligent, witty, and independent. She is also strong-willed and determined, often to the point of being stubborn. Her pride and prejudices often lead her into misunderstandings and conflicts with Mr. Darcy, but she eventually learns to see him in a more favorable light.
2. Jane Bennet - The middle Bennet sister, Jane is kind, gentle, and affectionate. She is also very sensitive and emotional, often tearfully expressing her feelings. While she is not as outspoken or assertive as her sister Elizabeth, she is a loyal and

### Adicionando o pipeline definido na langchain

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

# checking again that everything is working fine
print(llm(prompt="Compare the personalities of the Bennet sisters from Jane Austen's Pride and Prejudice."))

 Unterscheidung der Persönlichkeitsmerkmale der Bennet-Schwestern aus Jane Austens "Pride and Prejudice".

In Jane Austen's novel "Pride and Prejudice," the Bennet sisters are a lively and diverse group of women, each with their own unique personality traits. Here is a comparison of the personalities of the Bennet sisters:

1. Elizabeth Bennet - The eldest Bennet sister, Elizabeth is intelligent, witty, and independent. She is also strong-willed and determined, often to the point of being stubborn. Her pride and prejudices often lead her into misunderstandings and conflicts with Mr. Darcy, but she eventually learns to see him in a more favorable light.
2. Jane Bennet - The middle Bennet sister, Jane is kind, gentle, and affectionate. She is also very sensitive and emotional, often tearfully expressing her feelings. While she is not as outspoken or assertive as her sister Elizabeth, she is a loyal and supportive friend.
3. Mary Bennet - The middle Bennet sister, Mary is bookish, studiou

### Preparando os dados e o banco de vetores

In [ ]:
from langchain.document_loaders import WebBaseLoader

web_links = ["https://www.gutenberg.org/cache/epub/1342/pg1342.txt"]

loader = WebBaseLoader(web_links)
documents = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectorstore = FAISS.from_documents(all_splits, embeddings)

### Inicializando a chain

In [ ]:
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

### Conversando com os seus dados

In [ ]:
chat_history = []

query = "What are some themes in the story?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

 Great! Here are some possible themes that might emerge from the text:

1. Expectations vs. Reality: The story sets up expectations for the Gardiners' arrival, but the reality is quite different. Elizabeth is surprised to find her uncle alone, and the girls are excited to see their father but can't find him. This theme explores the idea of how our expectations don't always align with reality.
2. Family Dynamics: The story highlights the complexities of family dynamics, particularly between the Gardiners and the Bennetts. The girls' excitement to see their father is contrasted with their mother's worry and their uncle's absence. This theme could explore the tensions and relationships within families.
3. Social Hierarchy: The mention of Mr. Wickham's miniature on the mantelpiece suggests a social hierarchy. The Gardiners are upper class, while the Bennetts are lower class. This theme could explore the class distinctions in society and how they impact relationships.
4. Personal Growth: El

In [ ]:
chat_history = [(query, result["answer"])]

query = "Who is the author of pride and prejudice?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

  The author of Pride and Prejudice is Jane Austen.


In [ ]:
print(result['source_documents'])

[Document(page_content='The Project Gutenberg eBook of Pride and Prejudice\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\r\n\r\nTitle: Pride and Prejudice\r\n\r\n\r\nAuthor: Jane Austen\r\n\r\nRelease date: June 1, 1998 [eBook #1342]\r\n                Most recently updated: April 14, 2023\r\n\r\nLanguage: English\r\n\r\nCredits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The Internet Archive)\r\n\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***\r\n\r\n\r\n\r\n